# Drop out regularization
based on: Udemy DEEP LEARNING A-Z™ Course
Drop out is a solution for overfitting in ML.<br>
It shows itself as high variance in k-fold cross validation. <br>
It means some high accuracy and some low accuracy, result a high variance. <br>

## How it works:
At each epoch, the NN removes some of the nodes from the calculations. It will break down the correlations between the nodes and data and see if it still works. If so, those nodes will be removed finally, or get zero weights.<br>
To add it, we need to add it to be added to each layer. 

### Setup
To setup, we need to have these packages installed: <br>
1- Tensorflow<br>
2- Keras<br>
3- Theano | we will not work with this <br>

### A classification problem
based on the multi-dimensional data

In [15]:
import tensorflow as tf
# Theano is a mathematical library based on numpy, that can run on the GPU
#import theano
import keras

# we will need these for the ANN
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# will need this for data pre-processing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# will need this for splitting the data set
from sklearn.model_selection import train_test_split

# will need it for the scaling!
from sklearn.preprocessing import StandardScaler

# will need to calculate the loss
from sklearn.metrics import confusion_matrix

################################################################################
# will need it for the cross validation
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import cross_val_score

################################################################################
# will need it for the drop out
from keras.layers import Dropout

## Data pre-processing
Reading data from the CSV file and make it ready for the next steps.

### Imporet data

In [16]:
dataset = pd.read_csv('Churn_Modelling.csv')
# we only need features from 3 to 12 (upper bound is not inclusive)
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values
print (X[0])

[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]


### Encoding categorical data
There are two variables that are categorial, country and gender <br>
So, we convert them into one integer values! So if we apply it to country, it will replcae France, Germany, and Spain with 0,1,2! Nice!

In [17]:
# this will handle the country
labelEncoderX01 = LabelEncoder()
X[:,1] = labelEncoderX01.fit_transform(X[:,1])

# this will handle the gender
labelEncoderX02 = LabelEncoder()
X[:,2] = labelEncoderX02.fit_transform(X[:,2])

### Dummy coding
Now, we need to make some dummy coding! This will prevent the data to make this impression that categoprial values are actually impying any order. Spain is not higher or lower than France nor Germany.
We don't ned to do it with gender, since it only has two variables.
It basically make that 1,2,3 for countries into 3 different variables:
France  0, 0, 1
Germany 0, 1, 0
Spain   1, 0, 0
But we need to remove one of them, because we don't need the third one (if it is not German or French, then it is Spanish!)

In [18]:
oneHotEncoder = OneHotEncoder(categorical_features=[1])
X= oneHotEncoder.fit_transform(X).toarray()
X = X[:, 1:]

### Splitting the data into training and test
Easy peasy!

In [19]:
XTrain, XTest, yTrain, yTest = train_test_split(X,y, test_size = 0.2, random_state = 0)

### Feature Scaling
We must normalize the data, otherwise one feature is going to dominate the others! 

In [20]:
sc = StandardScaler()
# we first need to find the scale
XTrain = sc.fit_transform(XTrain)
# then we apply the same scale to the test set!
XTest = sc.transform(XTest)

# Cross Validation / K-Fold
This is a function that makes the ANN architecture like the previous section, but based on the Scikit_learn

In [25]:
def buildClassifier():
    classifier = Sequential()
    classifier.add(Dense(units= 6, kernel_initializer= "uniform",activation = "relu",input_dim= 11))
    # we start from 0.1 and then increase it by 0.1 to get out of overfitting, 
    # don't go beyond 0.5
    classifier.add(Dropout(rate=0.2))
    
    classifier.add(Dense(units= 6, kernel_initializer= "uniform",activation = "relu"))
    classifier.add(Dropout(rate = 0.2))
    
    classifier.add(Dense(units= 1, kernel_initializer= "uniform",activation = "sigmoid"))
    classifier.compile(optimizer = 'adam', loss ='binary_crossentropy',metrics = ['accuracy'])
    return classifier

Do not add n_jobs = -1 if you use GPU, it will use only one CUDA core!

In [26]:
%%time
classifier =  KerasClassifier(build_fn = buildClassifier, batch_size= 500, epochs = 100)
# returns the 10 accuracies from the 10 fold cross validation
acuracies = cross_val_score(estimator = classifier,X = XTrain, y = yTrain, cv = 10)#, n_jobs = -1) 

Epoch 1/100
7200/7200 [==============================] - 1s 137us/step - loss: 0.6930 - acc: 0.6983
Epoch 2/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6924 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6918 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6911 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6904 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6896 - acc: 0.7971
Epoch 7/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6888 - acc: 0.7971
Epoch 8/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6880 - acc: 0.7971
Epoch 9/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6872 - acc: 0.7971
Epoch 10/100
7200/7200 [==============================] - 0s 4us/step - loss: 0.6864 - acc: 0.7971
Epoch 11/100
7200

7200/7200 [==============================] - 0s 3us/step - loss: 0.4889 - acc: 0.7971
Epoch 84/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4861 - acc: 0.7971
Epoch 85/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4830 - acc: 0.7971
Epoch 86/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4798 - acc: 0.7971
Epoch 87/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4814 - acc: 0.7971
Epoch 88/100
7200/7200 [==============================] - 0s 2us/step - loss: 0.4789 - acc: 0.7971
Epoch 89/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4775 - acc: 0.7971
Epoch 90/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4783 - acc: 0.7971
Epoch 91/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4767 - acc: 0.7971
Epoch 92/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4731 - acc: 0.7971
Epoch 93/100
7200/7200 

7200/7200 [==============================] - 0s 3us/step - loss: 0.5942 - acc: 0.7967
Epoch 66/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5917 - acc: 0.7967
Epoch 67/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5888 - acc: 0.7967
Epoch 68/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5847 - acc: 0.7967
Epoch 69/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5818 - acc: 0.7967
Epoch 70/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5788 - acc: 0.7967
Epoch 71/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5765 - acc: 0.7967
Epoch 72/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5721 - acc: 0.7967
Epoch 73/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5688 - acc: 0.7967
Epoch 74/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5660 - acc: 0.7967
Epoch 75/100
7200/7200 

7200/7200 [==============================] - 0s 3us/step - loss: 0.6213 - acc: 0.7956
Epoch 48/100
7200/7200 [==============================] - 0s 2us/step - loss: 0.6180 - acc: 0.7956
Epoch 49/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6150 - acc: 0.7956
Epoch 50/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6097 - acc: 0.7956
Epoch 51/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6066 - acc: 0.7956
Epoch 52/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6026 - acc: 0.7956
Epoch 53/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5991 - acc: 0.7956
Epoch 54/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5940 - acc: 0.7956
Epoch 55/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5903 - acc: 0.7956
Epoch 56/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5860 - acc: 0.7956
Epoch 57/100
7200/7200 

7200/7200 [==============================] - 0s 3us/step - loss: 0.6691 - acc: 0.7975
Epoch 30/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6678 - acc: 0.7975
Epoch 31/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6664 - acc: 0.7975
Epoch 32/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6651 - acc: 0.7975
Epoch 33/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6636 - acc: 0.7975
Epoch 34/100
7200/7200 [==============================] - 0s 2us/step - loss: 0.6621 - acc: 0.7975
Epoch 35/100
7200/7200 [==============================] - 0s 2us/step - loss: 0.6605 - acc: 0.7975
Epoch 36/100
7200/7200 [==============================] - 0s 2us/step - loss: 0.6587 - acc: 0.7975
Epoch 37/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6574 - acc: 0.7975
Epoch 38/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6556 - acc: 0.7975
Epoch 39/100
7200/7200 

...

7200/7200 [==============================] - 0s 3us/step - loss: 0.5862 - acc: 0.7969
Epoch 58/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5818 - acc: 0.7969
Epoch 59/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5775 - acc: 0.7969
Epoch 60/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5746 - acc: 0.7969
Epoch 61/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5708 - acc: 0.7969
Epoch 62/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5678 - acc: 0.7969
Epoch 63/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5647 - acc: 0.7969
Epoch 64/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5596 - acc: 0.7969
Epoch 65/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5562 - acc: 0.7969
Epoch 66/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.5522 - acc: 0.7969
Epoch 67/100
7200/7200 

Epoch 39/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6478 - acc: 0.7963
Epoch 40/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6457 - acc: 0.7962
Epoch 41/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6431 - acc: 0.7963
Epoch 42/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6401 - acc: 0.7962
Epoch 43/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6375 - acc: 0.7963
Epoch 44/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6345 - acc: 0.7963
Epoch 45/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6318 - acc: 0.7962
Epoch 46/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6283 - acc: 0.7963
Epoch 47/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6258 - acc: 0.7963
Epoch 48/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6222 - acc: 0.7962
Epoch 49/1

7200/7200 [==============================] - 0s 3us/step - loss: 0.6774 - acc: 0.7957
Epoch 22/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6762 - acc: 0.7957
Epoch 23/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6752 - acc: 0.7957
Epoch 24/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6739 - acc: 0.7957
Epoch 25/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6726 - acc: 0.7957
Epoch 26/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6713 - acc: 0.7957
Epoch 27/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6701 - acc: 0.7957
Epoch 28/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6685 - acc: 0.7957
Epoch 29/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6669 - acc: 0.7957
Epoch 30/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6653 - acc: 0.7957
Epoch 31/100
7200/7200 

7200/7200 [==============================] - 0s 3us/step - loss: 0.6918 - acc: 0.7961
Epoch 4/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6912 - acc: 0.7961
Epoch 5/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6905 - acc: 0.7961
Epoch 6/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6898 - acc: 0.7961
Epoch 7/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6891 - acc: 0.7961
Epoch 8/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6883 - acc: 0.7961
Epoch 9/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6876 - acc: 0.7961
Epoch 10/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6868 - acc: 0.7961
Epoch 11/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6860 - acc: 0.7961
Epoch 12/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.6851 - acc: 0.7961
Epoch 13/100
7200/7200 [=====

7200/7200 [==============================] - 0s 3us/step - loss: 0.4900 - acc: 0.7961
Epoch 86/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4856 - acc: 0.7961
Epoch 87/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4858 - acc: 0.7961
Epoch 88/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4856 - acc: 0.7961
Epoch 89/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4835 - acc: 0.7961
Epoch 90/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4812 - acc: 0.7961
Epoch 91/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4771 - acc: 0.7961
Epoch 92/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4764 - acc: 0.7961
Epoch 93/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4750 - acc: 0.7961
Epoch 94/100
7200/7200 [==============================] - 0s 3us/step - loss: 0.4739 - acc: 0.7961
Epoch 95/100
7200/7200 

In [27]:
print(acuracies.mean())
print(acuracies.std())

0.796000009775
0.0101056905279


In [10]:
array([ 0.82875   ,  0.82625   ,  0.83250001,  0.82624999,  0.84875   ,
        0.83000001,  0.85250001,  0.81874999,  0.85250001,  0.84125002])
113s

array([ 0.78625   ,  0.79000002,  0.80000001,  0.78250003,  0.81625003,
        0.81      ,  0.78750002,  0.79624999,  0.79874998,  0.79500002])
33.1s

array([ 0.80624998,  0.79000002,  0.815     ,  0.815     ,  0.81625003,
        0.81      ,  0.83375001,  0.79374999,  0.81375003,  0.79500002])

77

0.796000009775
0.0101056905279

SyntaxError: invalid syntax (<ipython-input-10-22945615f28d>, line 3)